# Demonstrating the dataset generation for an ampl model

This notebook shows the generation of the dataset that are subsequently used to train a neural network for optimal control.

**Load dataset**

In [1]:
### %pylab notebook
from pyquad.ekin import *

datafile = 'datasets/hover_dataset.npz'
a=np.load(datafile)

input_labels = ['dx', 'dy', 'dz', 'vx', 'vy', 'vz', 'phi', 'theta', 'psi', 'p', 'q', 'r','omega', 'u', 'Mx_ext', 'My_ext', 'Mz_ext']
    
dataset = dict()
print('loading dataset...')
with np.load(datafile) as full_dataset:
    # total number of trajectories
    num = len(full_dataset['dx'])
    steps = full_dataset['dx'].shape[1]
    print(num, 'trajectories')
    dataset = {key: full_dataset[key] for key in input_labels}

reduced_dict = {}
desired_rows = 10000  # Number of rows to keep (10% of 100000)

for key, value in dataset.items():
    # Assuming `value` is a NumPy array of shape (100000, 199)
    if isinstance(value, np.ndarray):
        # Slice the array to keep only the first `desired_rows` rows
        if value.ndim == 2:
            reduced_value = value[:desired_rows, :]
        else:
            reduced_value = value[:desired_rows]
        reduced_dict[key] = reduced_value

dataset = reduced_dict

datafile_reduced = 'datasets/hover_dataset_reduced.npz'

np.savez_compressed(datafile_reduced, **dataset)

loading dataset...
100000 trajectories


**View trajectories**

In [2]:
def transform_back(dx,dy,dz,vx,vy,vz,phi,theta,psi):
    Rx = np.array([
        [1, 0, 0],
        [0, np.cos(phi), -np.sin(phi)],
        [0, np.sin(phi), np.cos(phi)]
    ])
    Ry = np.array([
        [np.cos(theta), 0, np.sin(theta)],
        [0, 1, 0],
        [-np.sin(theta), 0, np.cos(theta)]
    ])
    Rz = np.array([
        [np.cos(psi), -np.sin(psi), 0],
        [np.sin(psi), np.cos(psi), 0],
        [0, 0, 1]
    ])
    R = Rz@Ry@Rx
    x_new, y_new, z_new = -R@[dx, dy, dz]
    vx_new, vy_new, vz_new = R@[vx, vy, vz]
    return x_new, y_new, z_new, vx_new, vy_new, vz_new, phi, theta, psi

In [5]:
%matplotlib widget
a = np.load(datafile)
num = a['dx'].shape[0]
print(num)

t,dx,dy,dz,vx,vy,vz,phi,theta,psi,u = (a[key][10000:10001] for key in 't dx dy dz vx vy vz phi theta psi u'.split(' '))

100000


In [8]:
from quadcopter_animation import animation
import importlib
importlib.reload(animation)

x,y,z,vx,vy,vz,phi,theta,psi = np.vectorize(transform_back)(dx,dy,dz,vx,vy,vz,phi,theta,psi)
# t,x,y,z,vx,vy,vz,phi,theta,psi,u = (a[key] for key in 't x y z vx vy vz phi theta psi u'.split(' '))

animation.animate(t, x, y, z, phi, theta, psi, u, multiple_trajectories=True, simultaneous=False, step=1, waypoints=[np.array([0,0,0])], colors=[(255,0,0)]*100)